In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from  pyspark.sql import SQLContext
from pyspark.sql.types import *


In [ ]:
spark = SparkSession.builder \
             .master("local") \
             .appName("stream to csv")\
             .config("spark.some.config.option", "some-value") \
            .getOrCreate()


In [ ]:
dataSchema = StructType([StructField('creation_time',TimestampType(),True),
        StructField('key',StringType(),True),
        StructField('x',LongType(),True),
                        ])

In [ ]:
streaming1 = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
.csv( "input_stream_csv1")
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [ ]:
streaming1_with_impressions = streaming1.withWatermark("creation_time", "2 minutes")

In [ ]:
streaming2 = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
.csv( "input_stream_csv2")
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [ ]:
streaming1.registerTempTable("my_table1")
streaming2.registerTempTable("my_table2")


In [ ]:
spark.sql("""select t1.creation_time, t1.x as x1, t2.x as x2 from my_table1 t1 
inner join my_table2 t2 on t1.key = t2.key
where t1.creation_time > current_timestamp() - interval 3 minutes""")\
.writeStream.trigger(processingTime='10 seconds')\
.format("parquet")\
.option("checkpointLocation", "checkpoint_dir").outputMode("append")\
.option("path", "stream_dir5").start()

In [ ]:
df = spark.read.parquet("stream_dir5/")
df.registerTempTable("result")
spark.sql("""select * from result
""").show()
